## Import Necessary Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from datetime import datetime
import seaborn as sns
import os

## Extract Data

In [2]:
inspections = pd.read_csv('../rawdata/inspections.csv')
violations = pd.read_csv('../rawdata/violations.csv')

## Data Exploration

In [3]:
inspections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386336 entries, 0 to 386335
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   ACTIVITY DATE         386336 non-null  object
 1   OWNER ID              386336 non-null  object
 2   OWNER NAME            386336 non-null  object
 3   FACILITY ID           386336 non-null  object
 4   FACILITY NAME         386336 non-null  object
 5   RECORD ID             386336 non-null  object
 6   PROGRAM NAME          386336 non-null  object
 7   PROGRAM STATUS        386336 non-null  object
 8   PROGRAM ELEMENT (PE)  386336 non-null  int64 
 9   PE DESCRIPTION        386336 non-null  object
 10  FACILITY ADDRESS      386336 non-null  object
 11  FACILITY CITY         386336 non-null  object
 12  FACILITY STATE        386336 non-null  object
 13  FACILITY ZIP          386336 non-null  int64 
 14  SERVICE CODE          386336 non-null  int64 
 15  SERVICE DESCRIPTI

In [4]:
violations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   SERIAL NUMBER          1048575 non-null  object 
 1   VIOLATION  STATUS      1048575 non-null  object 
 2   VIOLATION CODE         1048575 non-null  object 
 3   VIOLATION DESCRIPTION  1048575 non-null  object 
 4   POINTS                 1048575 non-null  float64
dtypes: float64(1), object(4)
memory usage: 40.0+ MB


In [5]:
inspections.columns = inspections.columns.str.lower()
inspections.columns = inspections.columns.str.replace(' ','_')
violations.columns = violations.columns.str.lower()
violations.columns = violations.columns.str.replace(' ','_')

In [6]:
inspections.score.max(), inspections.score.min()

(100, 3)

In [7]:
violations.points.max(), violations.points.min()

(11.0, 0.0)

* Inspections Score MAX value shows us there are some restaurants that got perfect scores

In [8]:
inspections['serial_number'].nunique(),violations['serial_number'].nunique()

(386336, 224009)

In [9]:
merged_df = pd.merge(inspections, violations, how = 'left', on = 'serial_number')

In [10]:
merged_df.head()

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,facility_zip,service_code,service_description,score,serial_number,employee_id,violation__status,violation_code,violation_description,points
0,11/10/2015,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,90505,1,ROUTINE INSPECTION,93,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F027,# 27. Food separated and protected,1.0
1,11/10/2015,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,90505,1,ROUTINE INSPECTION,93,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F037,# 37. Adequate ventilation and lighting; desig...,1.0
2,11/10/2015,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,90505,1,ROUTINE INSPECTION,93,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F040,"# 40. Plumbing: Plumbing in good repair, prope...",1.0
3,11/10/2015,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,90505,1,ROUTINE INSPECTION,93,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F006,# 06. Adequate handwashing facilities supplied...,2.0
4,11/10/2015,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,90505,1,ROUTINE INSPECTION,93,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F036,"# 36. Equipment, utensils and linens: storage ...",1.0


In [11]:
merged_df['activity_date'] = pd.to_datetime(merged_df['activity_date'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1210653 entries, 0 to 1210652
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   activity_date          1210653 non-null  datetime64[ns]
 1   owner_id               1210653 non-null  object        
 2   owner_name             1210653 non-null  object        
 3   facility_id            1210653 non-null  object        
 4   facility_name          1210653 non-null  object        
 5   record_id              1210653 non-null  object        
 6   program_name           1210653 non-null  object        
 7   program_status         1210653 non-null  object        
 8   program_element_(pe)   1210653 non-null  int64         
 9   pe_description         1210653 non-null  object        
 10  facility_address       1210653 non-null  object        
 11  facility_city          1210653 non-null  object        
 12  facility_state         12106

### Understanding the Scoring
* Points column is the number of points taken for the violation.
* Score is the overall inpsection score out of 100.
---
* Information from LA County Food Inspection Reference Guide:
__[Inspection Guide](http://www.publichealth.lacounty.gov/eh/docs/RefGuideFoodInspectionReport.pdf)__

>* Major Violation results in 4 pt Deduction
>* Two Major Violations result in additional 3pt deduction and possible closure
>* Permit suspension due to vermin infestation, no water, or now sewage available, additional 7pt deduction for each closure type
>* Minor Violation result in 2pt deduction
>* Good Retail Practicies (Low Risk Violations), result in 1 pt violation

### Additional Notes

* Serial Number is unique ID for each inspection
* Record ID is a UNIQUE ID number for each health program at a facility. A facility can have multiple programs!
* Program name is a unique name for each program

* Now restaurants that got perfect scores are included in the merged data frame --> look at null values in violation_status, violation_code, violation_description, and points
* Split the program elements part!
* Analyze by program elements broken down
* Break down the date time by year --> analyze by year
* Analyze by month!

* Now inspect score and number of inspections by year and month
* First remove duplicates --> serial numbers change with each inspection --> remove duplicates for that to get the accurate count and distribution for scores

In [12]:
merged_df['year'] = merged_df['activity_date'].dt.year
merged_df['month'] = merged_df['activity_date'].dt.month
merged_df['day'] = merged_df['activity_date'].dt.day
merged_df['quarter'] = merged_df['activity_date'].dt.quarter
merged_df.head()

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,serial_number,employee_id,violation__status,violation_code,violation_description,points,year,month,day,quarter
0,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F027,# 27. Food separated and protected,1.0,2015,11,10,4
1,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F037,# 37. Adequate ventilation and lighting; desig...,1.0,2015,11,10,4
2,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F040,"# 40. Plumbing: Plumbing in good repair, prope...",1.0,2015,11,10,4
3,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F006,# 06. Adequate handwashing facilities supplied...,2.0,2015,11,10,4
4,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA0EU4KUG,EE0000306,OUT OF COMPLIANCE,F036,"# 36. Equipment, utensils and linens: storage ...",1.0,2015,11,10,4


### Categorize Violations as Major, Minor, Critical using Points and Violation Code

Match point value with violation codes based on the reference book above. 
* Critical -> 11 pts deduction
* Major --> 4 pts deduction
* Minor --> 2 pts deduction
* Retail Practices --> 1 pt deduction
* Double Major --> 3pt reduction
* NaN for all that don't match conditions

In [13]:
merged_df['points'].value_counts(dropna = False)

1.0     792962
2.0     187705
NaN     162415
4.0      48694
0.0      14366
3.0       2324
11.0      2187
Name: points, dtype: int64

In [14]:
zero = merged_df[merged_df['points'] == 0.0]
zero[zero['violation_code']=='F021']

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,serial_number,employee_id,violation__status,violation_code,violation_description,points,year,month,day,quarter
205819,2016-06-07,OW0030630,ROMAN INC,FA0037751,MARISCOS SAN ISIDRO,PR0001060,MARISCOS SAN ISIDRO,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DAH5EMEOW,EE0000592,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2016,6,7,2
289189,2017-01-19,OW0022348,LOVE SUSHI II INC.,FA0036104,LOVE SUSHI,PR0021087,LOVE SUSHI,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,DAEZVGAFY,EE0000364,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,1,19,1
334406,2017-03-09,OW0226883,MARICELA MEDINA,FA0221954,LITO'S CORK ROOM,PR0176941,LITO'S CORK ROOM,ACTIVE,1633,RESTAURANT (31-60) SEATS LOW RISK,...,DA0801180,EE0000081,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,3,9,1
346355,2017-03-20,OW0035201,"TECHAWUTIWAT, TANAPORN",FA0065827,LITTLE JASMINE THAI CUISINE,PR0017693,LITTLE JASMINE THAI CUISINE,INACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,DAMPBDTVI,EE0000067,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,3,20,1
349721,2017-03-23,OW0008418,"COBAR, RENE GUILLERMO",FA0002904,ADDY'S BAKERY,PR0017304,ADDY'S BAKERY,INACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,DAMHQ7JGY,EE0001009,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,3,23,1
351492,2017-02-21,OW0186096,GREGORIA ESCARENO,FA0180753,EL MARATIAL,PR0174555,EL MANANTIAL,INACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,DALFSRGXD,EE0000691,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,2,21,1
356004,2017-02-27,OW0178492,ALEJANDRO GERECERO ORTIZ,FA0171405,EL PESCADOR #11,PR0161474,EL PESCADOR #11,INACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DA110CDDI,EE0000145,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,2,27,1
357453,2017-04-04,OW0015476,HI WORLD CORPORATION,FA0038760,TSUJITA LA RESTAURANT,PR0031994,TSUJITA LA RESTAURANT,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,DAEVNWEA5,EE0000684,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,4,4,2
361713,2017-02-23,OW0185252,LAURA VILLAVICENCIO,FA0179759,LA MONARCA MICHOACANA,PR0173419,LA MONARCA MICHOACANA,ACTIVE,1631,RESTAURANT (0-30) SEATS MODERATE RISK,...,DA7KIJ0E0,EE0000691,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,2,23,1
363052,2017-02-23,OW0186222,TAQUERIA LOS SANTOS,FA0180884,TAQUERIA LOS SANTOS,PR0174716,TAQUERIA LOS SANTOS,ACTIVE,1632,RESTAURANT (0-30) SEATS HIGH RISK,...,DAINGOISX,EE0000113,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2017,2,23,1


In [15]:
merged_df[merged_df['serial_number']=='DAH5EMEOW']
# 0.0 points with a violation --> duplicate values!!!

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,serial_number,employee_id,violation__status,violation_code,violation_description,points,year,month,day,quarter
205817,2016-06-07,OW0030630,ROMAN INC,FA0037751,MARISCOS SAN ISIDRO,PR0001060,MARISCOS SAN ISIDRO,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DAH5EMEOW,EE0000592,OUT OF COMPLIANCE,F014,# 14. Food contact surfaces: clean and sanitized,4.0,2016,6,7,2
205818,2016-06-07,OW0030630,ROMAN INC,FA0037751,MARISCOS SAN ISIDRO,PR0001060,MARISCOS SAN ISIDRO,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DAH5EMEOW,EE0000592,OUT OF COMPLIANCE,F021,# 21b. Water available,4.0,2016,6,7,2
205819,2016-06-07,OW0030630,ROMAN INC,FA0037751,MARISCOS SAN ISIDRO,PR0001060,MARISCOS SAN ISIDRO,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DAH5EMEOW,EE0000592,OUT OF COMPLIANCE,F021,# 21b. Water available,0.0,2016,6,7,2
205820,2016-06-07,OW0030630,ROMAN INC,FA0037751,MARISCOS SAN ISIDRO,PR0001060,MARISCOS SAN ISIDRO,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,DAH5EMEOW,EE0000592,OUT OF COMPLIANCE,F052,# 01b. Food safety certification,2.0,2016,6,7,2


In [16]:
crit_code = ['F021','F022','F023']
maj_code = ['F002','F005','F007','F008','F009','F010','F011','F013','F014','F015','F020','F021','F022','F023','F054']
min_code = maj_code + ['F001','F003','F004','F006','F012','F016','F017','F018','F019','F022','F023','F052']
ret_code = ['F024','F025','F026','F027','F028','F029','F030','F031','F032','F033','F034','F035','F036','F037','F038',
            'F039','F040','F041','F042','F043','F044','F045','F046','F048']

In [17]:
conditions = [merged_df['violation_code'].isin(crit_code) & (merged_df['points'] == 11.0),\
              merged_df['violation_code'].isin(maj_code) & (merged_df['points'] == 4.0),\
              merged_df['violation_code'].isin(min_code) & (merged_df['points'] == 2.0),\
              merged_df['violation_code'].isin(ret_code) & (merged_df['points'] == 1.0),\
              merged_df['violation_code'].isin(maj_code) & (merged_df['points'] == 3.0)]

merged_df['violation_category'] = np.select(conditions, [5,3,2,1,4], default = np.nan)
# 5 for Critical Violation, 4 Double Major Violation, 3 for Major Violation, 2 for Minor Violation, 1 for Retail Violation

In [18]:
merged_df.head()

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,employee_id,violation__status,violation_code,violation_description,points,year,month,day,quarter,violation_category
0,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EE0000306,OUT OF COMPLIANCE,F027,# 27. Food separated and protected,1.0,2015,11,10,4,1.0
1,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EE0000306,OUT OF COMPLIANCE,F037,# 37. Adequate ventilation and lighting; desig...,1.0,2015,11,10,4,1.0
2,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EE0000306,OUT OF COMPLIANCE,F040,"# 40. Plumbing: Plumbing in good repair, prope...",1.0,2015,11,10,4,1.0
3,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EE0000306,OUT OF COMPLIANCE,F006,# 06. Adequate handwashing facilities supplied...,2.0,2015,11,10,4,2.0
4,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,EE0000306,OUT OF COMPLIANCE,F036,"# 36. Equipment, utensils and linens: storage ...",1.0,2015,11,10,4,1.0


### Categorize Based on Breakdown of pe_description
* Breakdown pe_description to facility type, size, risk level
* Use International Building Code guidelines to Standardize Capacity --> convert seat value to square feet
* 15 sq ft dedicated to one occupant
* Remove non-restaurant, non-caterer and food market retail??

In [19]:
merged_df['pe_description'].value_counts()

RESTAURANT (0-30) SEATS HIGH RISK                276881
RESTAURANT (31-60) SEATS HIGH RISK               224055
RESTAURANT (61-150) SEATS HIGH RISK              180171
RESTAURANT (0-30) SEATS MODERATE RISK            171401
FOOD MKT RETAIL (1-1,999 SF) LOW RISK             74607
RESTAURANT (151 + ) SEATS HIGH RISK               69592
FOOD MKT RETAIL (1-1,999 SF) HIGH RISK            50050
RESTAURANT (31-60) SEATS MODERATE RISK            33206
FOOD MKT RETAIL (2,000+ SF) LOW RISK              32242
RESTAURANT (61-150) SEATS MODERATE RISK           23846
RESTAURANT (0-30) SEATS LOW RISK                  22133
FOOD MKT RETAIL (2,000+ SF) HIGH RISK             19881
FOOD MKT RETAIL (1-1,999 SF) MODERATE RISK        10739
RESTAURANT (31-60) SEATS LOW RISK                  7109
FOOD MKT RETAIL (2,000+ SF) MODERATE RISK          5874
RESTAURANT (151 + ) SEATS MODERATE RISK            3914
RESTAURANT (61-150) SEATS LOW RISK                 2832
RESTAURANT (151 + ) SEATS LOW RISK              

In [20]:
# Categorize on Facility Type - Restaurant, Market Retail etc
merged_df['facility_type'] = merged_df.pe_description.str.split('(').str[0]

# Categorize based on risk level - High = 3 , Medium = 2, Low = 1
# Risk is categorized based on type of food prep they do. More complex = Higher Risk
lst = ['HIGH','MODERATE','MOD','LOW']
def fun(a):
    for i in lst:
        if i in a:
            return i
    return np.nan
merged_df['risk_level'] = merged_df.pe_description.apply(lambda x: fun(x))
mapping = {'HIGH': '3', 'MODERATE':'2','MOD':'2', 'LOW':'1', np.nan: np.nan} 
merged_df['risk_level'] = merged_df['risk_level'].replace(mapping)
merged_df['risk_level'].value_counts(dropna = False)

3.0    820938
2.0    248985
1.0    139898
NaN       832
Name: risk_level, dtype: int64

In [21]:
merged_df.head()

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,...,violation_code,violation_description,points,year,month,day,quarter,violation_category,facility_type,risk_level
0,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,F027,# 27. Food separated and protected,1.0,2015,11,10,4,1.0,RESTAURANT,3.0
1,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,F037,# 37. Adequate ventilation and lighting; desig...,1.0,2015,11,10,4,1.0,RESTAURANT,3.0
2,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,F040,"# 40. Plumbing: Plumbing in good repair, prope...",1.0,2015,11,10,4,1.0,RESTAURANT,3.0
3,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,F006,# 06. Adequate handwashing facilities supplied...,2.0,2015,11,10,4,2.0,RESTAURANT,3.0
4,2015-11-10,OW0126532,GEN TORRANCE LLC,FA0162834,GEN KOREAN BBQ,PR0150985,GEN KOREAN BBQ,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,...,F036,"# 36. Equipment, utensils and linens: storage ...",1.0,2015,11,10,4,1.0,RESTAURANT,3.0


In [22]:
import regex
import re
def cap_find(row):
    if 'SEATS' in row:
        seat_range = regex.split('\(|\)',row)[1]
        if '+' in seat_range:
            seat_conv = seat_range.replace('+','')
            return int(seat_conv)*15
        if '-' in seat_range:
            seat_conv = seat_range.split('-')[1]
            return int(seat_conv)*15
    elif 'SQ. FT.' in row or 'SF' in row:
        bracket_info = regex.split('\(|\)',row)[1]
        if '+' in bracket_info:
            b_info = bracket_info.replace(',','').split()[0]
            return int(b_info.replace('+',''))
        if '-' in bracket_info:    
            brkt_info = bracket_info.split()[0].replace(',','')
            return int(brkt_info.split('-')[1])
    else:
        return np.nan
    
merged_df['sq_ft_max'] = merged_df['pe_description'].apply(lambda row: cap_find(row))
merged_df['sq_ft_max'].value_counts()

450.0      470415
900.0      264370
2250.0     206849
1999.0     135787
2265.0      74481
2000.0      57998
9999.0        257
999.0         151
4999.0         84
5999.0         35
10000.0        25
3999.0         21
6000.0         14
5000.0          7
Name: sq_ft_max, dtype: int64

### Drop Duplicates under following conditions
If point value is 0.0 and score does NOT equal 100, drop the value

In [23]:
dropped_duplicates = merged_df.loc[~((merged_df['points']==0.0) & (merged_df['score'] != 100)),:]

In [24]:
# Still have NaN values for points which results in NaN for Violation Category, Violation Code, Status and Description
dropped_duplicates['points'].value_counts(dropna=False)

1.0     792962
2.0     187705
NaN     162415
4.0      48694
3.0       2324
11.0      2187
0.0        109
Name: points, dtype: int64

In [25]:
dropped_duplicates[dropped_duplicates['points'].isna()].head().iloc[:,0:20]

,activity_date,owner_id,owner_name,facility_id,facility_name,record_id,program_name,program_status,program_element_(pe),pe_description,facility_address,facility_city,facility_state,facility_zip,service_code,service_description,score,serial_number,employee_id,violation__status
8,2015-11-24,OW0035173,TEA LAB INC.,FA0050882,TEA LAB,PR0048348,TEA LAB,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,18912 E GALE AVE STE #A,ROWLAND HEIGHTS,CA,91748,1,ROUTINE INSPECTION,91,DAPTZRYA2,EE0000487,NaN
14,2015-12-01,OW0028792,"POLLY'S PIES, INC",FA0068859,POLLY'S PIES,PR0004952,POLLY'S PIES,ACTIVE,1638,RESTAURANT (61-150) SEATS HIGH RISK,17198 S NORWALK BLVD,CERRITOS,CA,90703,1,ROUTINE INSPECTION,96,DARLLFGYK,EE0000260,NaN
15,2015-12-02,OW0021804,"LIM, NHANG",FA0059721,CHINATOWN EXPRESS,PR0047596,CHINATOWN EXPRESS,INACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,1320 N VINE ST,LOS ANGELES,CA,90038,1,ROUTINE INSPECTION,93,DATEOAEDT,EE0000709,NaN
16,2015-12-03,OW0022909,"MAHMOUD,ABDELLATIF B",FA0041578,MOUN OF TUNIS,PR0019819,MOUN OF TUNIS,ACTIVE,1635,RESTAURANT (31-60) SEATS HIGH RISK,7445 SUNSET BLVD # B,LOS ANGELES,CA,90046,1,ROUTINE INSPECTION,94,DAW3AOIVL,EE0000828,NaN
17,2015-12-08,OW0033775,SPECIALTY RESTAURANTS CORP,FA0059347,CASTAWAY,PR0006356,CASTAWAY,ACTIVE,1641,RESTAURANT (151 + ) SEATS HIGH RISK,1250 E HARVARD RD,BURBANK,CA,91501,1,ROUTINE INSPECTION,95,DAQVKA2TS,EE0000025,NaN


In [26]:
#Cannot Just drop these rows because there are many rows with NaN value for points
dropped_duplicates[dropped_duplicates['serial_number'] == 'DATEOAEDT'].iloc[:,15:]

,service_description,score,serial_number,employee_id,violation__status,violation_code,violation_description,points,year,month,day,quarter,violation_category,facility_type,risk_level,sq_ft_max
15,ROUTINE INSPECTION,93,DATEOAEDT,EE0000709,NaN,NaN,NaN,NaN,2015,12,2,4,NaN,RESTAURANT,3.0,900.0


In [28]:
dropped_duplicates.shape

(1196396, 31)

### Latitude and Longitude

In [ ]:


locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('24301 CRENSHAW BLVD,TORRANCE,90505')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

In [ ]:
# Concatenate Address for geopy Use
concat_address = dropped_duplicates.loc[:,'facility_address'] +',' + dropped_duplicates.loc[:,'facility_city'] +','+ dropped_duplicates.loc[:,'facility_zip'].astype(str) 
dropped_duplicates = pd.concat([dropped_duplicates, concat_address.rename('concat_address')], axis = 1)
dropped_duplicates.head()

In [ ]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent='myGeocoder')

from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

dropped_duplicates['location'] = dropped_duplicates['concat_address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
dropped_duplicates['point'] = dropped_duplicates['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
dropped_duplicates[['latitude', 'longitude', 'altitude']] = pd.DataFrame(dropped_duplicates['point'].tolist(), index=dropped_duplicates.index)

In [30]:
dropped_duplicates.facility_address.nunique()

43618

### Drop Duplicates

In [ ]:
# Serial numbers are unique for each inspection. 
dropped_dup = merged_df.drop_duplicates(subset = ['serial_number'], keep = 'first')
dropped_dup.info()

### Notes
* There were 386336 inspections since 2015
* There are some missing violation information - Points, Violation_status, violation_code, violation_description

### Number of Inspections Yearly, Monthly, and by Day of Month

In [ ]:
fig, ax = plt.subplots(3, 1, figsize = (10,20))

for idx, time in enumerate(['year','month','day']):

    ct = dropped_dup[time].value_counts().sort_index()
    
    ax[idx] = sns.barplot(x = ct.index, y = ct.values, order = ct.index, ax = ax[idx])
    
    ax[idx].set_xlabel(time)
    ax[idx].set_ylabel('Frequency')
    ax[idx].set_title('Inspection Frequency {}'.format(time))
    
    rects = ax[idx].patches
    labels = ct.values
    
    for rect, label in zip(rects,labels):
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_height() + 5
        ax[idx].text(x,y,label, ha = 'center', va = 'bottom')
    
plt.show()
    

### Quartely and Monthly Inspection Count by Year

In [ ]:
#Quarterly Inspection Count by Year

fig, ax = plt.subplots(6,1, figsize = (10,20))
years = [2015,2016,2017,2018,2019,2020]

for idx, year in enumerate(years):

    cnt = dropped_dup[dropped_dup['year']==year]['quarter'].value_counts().sort_index()
    
    ax[idx] = cnt.plot.bar(ax = ax[idx])
    
    ax[idx].set_xlabel(time)
    ax[idx].set_ylabel('Frequency')
    ax[idx].set_title('Inspection Frequency by Quarter for {}'.format(year))
    fig.tight_layout(pad = 0.3)
    
    rects = ax[idx].patches
    labels = cnt.values
    
    for rect, label in zip(rects,labels):
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_height() + 5
        ax[idx].text(x,y,label, ha = 'center', va = 'bottom')
        
plt.show()

In [ ]:
#Monthly Inspection Count by Year

fig, ax = plt.subplots(6,1, figsize = (10,20))
years = [2015,2016,2017,2018,2019,2020]

for idx, year in enumerate(years):

    cnt = dropped_dup[dropped_dup['year']==year]['month'].value_counts().sort_index()
    
    ax[idx] = cnt.plot.bar(ax = ax[idx])
    
    ax[idx].set_xlabel('Month')
    ax[idx].set_ylabel('Frequency')
    ax[idx].set_title('Inspection Frequency by Month for {}'.format(year))
    fig.tight_layout(pad = 0.3)
    
    rects = ax[idx].patches
    labels = cnt.values
    
    for rect, label in zip(rects,labels):
        x = rect.get_x() + rect.get_width()/2
        y = rect.get_height() + 5
        ax[idx].text(x,y,label, ha = 'center', va = 'bottom')
        
plt.show()

* Zip Analysis with duplicates removed!

### Score Distribution on Dropped Duplicate Serial Numbers

In [ ]:
fig, ax = plt.subplots(6,1, figsize = (5,20))
years = [2015,2016,2017,2018,2019,2020]

for idx, year in enumerate(years):

    data = dropped_dup[dropped_dup['year']==year]
    sns.boxplot(x='quarter',y='score',data = data,ax = ax[idx])
    
   
    ax[idx].set_xlabel('Quarter')
    ax[idx].set_ylabel('Score')
    ax[idx].set_title('Quarterly Score Distribution for {}'.format(year))
    fig.tight_layout(pad = 0.3)
    
       
plt.show()

In [ ]:
dropped_dup.groupby(['year','quarter'])['score'].describe()

In [ ]:
fig, ax = plt.subplots(6,1, figsize = (5,20))
years = [2015,2016,2017,2018,2019,2020]

for idx, year in enumerate(years):

    data = dropped_dup[dropped_dup['year']==year]
    sns.boxplot(x='month',y='score',data = data,ax = ax[idx])
    
   
    ax[idx].set_xlabel('Month')
    ax[idx].set_ylabel('Score')
    ax[idx].set_title('Monthly Score Distribution for {}'.format(year))
    fig.tight_layout(pad = 0.3)
    
       
plt.show()

In [ ]:
dropped_dup.groupby(['year','month'])['score'].describe()

* 2020 might have a transcription error with a score value of 3.
* Use heatmap to see if there is a strong correlation with year, months, and  days
*

### Score Distrubution by Zip

In [ ]:
zip_split = dropped_dup.groupby('facility_zip')['score'].mean().sort_values(ascending=False)
zip_split

In [ ]:
sns.kdeplot(data=zip_split)
plt.xlabel('Score')
plt.ylabel('Density')
plt.legend([],[], frameon=False)

### Score Distribution by Program Element - Dupllicate Removed

In [ ]:
plt.figure(figsize=(20,10))

sns.boxplot(x = 'score',y = 'pe_description', data = dropped_dup, orient = 'h')

### Score Distribution by Violation Type - Duplicate NOT Removed

### Explore violation data

In [ ]:
merged_df['violation__status'].value_counts()

In [ ]:
merged_df[merged_df['violation__status']=='VIOLATION']

In [ ]:
merged_df['violation_code'].unique()

* Check out F002, F005, F007-F011, F013-F15, F20-23 and F50 which is permit suspension

In [ ]:
lst = ['F002','F005','F007','F008','F009','F010','F011','F013','F014','F015','F020','F021','F022','F023','F050']


b = pd.DataFrame()
for i in lst:
    a = merged_df[merged_df['violation_code']==i].head(2)
    b = b.append(a)
b

* Filter Major Violations using pts
---
#### Major Violations 
* Major Violations have a point deduction of 4pts or 11pts. 
* The violation code is lst above. --> Some violation codes can be marked as minor violations hence the conditional loop.

In [ ]:
major_violations = merged_df.loc[(merged_df['points'] == 4.0) | (merged_df['points'] == 11.0)]
major_violations.shape

In [ ]:
major_violations.hist(column = 'score', bins = 20)

In [ ]:
sns.kdeplot(data = major_violations,x='score', cut = 0)

In [ ]:
major_violations['facility_zip'].value_counts().head(15)

In [ ]:
(major_violations['record_id'] + ' : ' + major_violations['facility_name']).value_counts().head(30)

In [ ]:
merged_df[(merged_df['record_id'] == 'PR0189419') & (merged_df['year']==2018)]

In [ ]:
plt.figure(figsize=(40,10))

sns.boxplot(x = 'score',y = 'violation_description', data = merged_df, orient = 'h')

### Major Risks and Closure

### Categorizing Critical Risk (Minor & Major), Good Retail Practices, and Compliance Violations

In [ ]:
merged_df['points'].value_counts(dropna=False)

### Notes about points
* 4 pts is MAJOR VIOLATION
* 3 pts is the additional reduction for two major violations
* 2 pts is a Minor Violation
* 1 pt is Good Retail Practices Violation
* Permit suspension due to vermin infestation, no water, or now sewage available, additional 7pt deduction for each closure type -> 4 + 7 = 11 so 11 pts must be one of those
* What are the NaN Values and What are the 0.0 Values?



In [ ]:
zero_pts = merged_df.loc[merged_df['points']==0.0]
zero_pts.iloc[:,0:18].head()


In [ ]:
null_pts = merged_df[merged_df['points'].isnull()]
null_pts['score'].value_counts().head()

In [ ]:
perf_score = merged_df[merged_df['score']==100]
perf_score['points'].value_counts(dropna=False)

* Null doesn't necessarily mean that the restaurants got perfect sores
* Score of 100 doesn't necessarily mean that the points is 0 or Nan......
* So there is some transcription error.....

* How do I deal with this??? ASK MENTOR!!!

In [ ]:
print(merged_df[(merged_df['points']==2) &(merged_df['score']==100)])
merged_df[merged_df['serial_number']=='DABH9GD40']

In [ ]:
sns.kdeplot(data = dropped_dup, x = 'score', hue = 'pe_description')

###  Look at Score Distribution on Dropped Duplicate Serial Number!

In [ ]:
merged_df[('violation_code','violation_id')].unique()